In [1]:
# !pip install xgboost

# Data Loading

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import Dataset
import numpy as np
import nibabel as ni
import os, shutil
import time
import random
import pandas as pd 
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
import loss
# from model_VAE import VAE
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math


In [23]:
class MRI_dataloader(Dataset):
    def __init__(self, path, img_size=(137, 128, 128)):
        self.path = path
        self.filenames = [i for i in os.listdir(path) if i.endswith(".nii")]
        #self.upsample = torch.nn.Upsample(size=(137, 128, 128), mode='trilinear', align_corners=True)
        self.img_size = img_size
        
    def __getitem__(self, index):
        image = ni.load(os.path.join(self.path, self.filenames[index]))
        image = np.array(image.dataobj)
        image = np.moveaxis(image, [0, 1], [1, 0])
        img_scale = tuple([self.img_size[i]/image.shape[i] for i in range(len(image.shape))])
        image = zoom(image, img_scale)
#         image = image / 255.
        image = torch.from_numpy(image)
        image = image.unsqueeze(0)
        #image = torch.permute(image, (1, 0, 2)).unsqueeze(0)
        return image
    
    def __len__(self):
        return len(self.filenames)

In [24]:
# from xgboost import XGBRegressor


device = 'cuda'

# ckpt_path_VAE = './checkpoint/best_model.pt' 
# ckpt_path_AE = './checkpoint/best_model_AE.pt'

ckpt_path_AE = './checkpoint/best_model_AE_l1_loss.pt'
ckpt_path_VAE = './checkpoint/best_model_vae_l1.pt'

model_AE = torch.load(ckpt_path_AE).to(device)
# model_VAE = torch.load(ckpt_path_VAE).to(device)


# ckpt_path = path2save = "./model_VAE.pt"
# ckpt_path = ckpt_path.format(52)
imagePath = 'dataset/images'
mri_loader = MRI_dataloader(imagePath)
batch_size = 1
dataloader = DataLoader(mri_loader, batch_size=batch_size, shuffle=True)
# model = VAE().to(device)

import pandas as pd

labels = pd.read_csv('dataset/data.csv')

model_AE.eval()
for batch in tqdm(dataloader):
    batch = batch.to(device)
    # print(model_AE(batch))
    z, y = model_AE(batch)
    print(y.shape)

    break

  0%|                                                                                                                                                     | 0/171 [00:00<?, ?it/s]

torch.Size([1, 512])


In [25]:
# out = model_AE(batch)
# print(len(out))
# out.shape

In [26]:
# model_AE

In [27]:
latent_dim = 512
epsilon = torch.normal(size=(1, latent_dim), mean=0, std=1.0, device=device)
# z = z_mean + z_log_sigma.exp()*epsilon

# Preprocessing

In [28]:
from scipy.ndimage import zoom
import nibabel as ni

# def preprocess_image(path, filename, img_size=(137, 128, 128)):
#     image = ni.load(os.path.join(path, filename))
#     image = np.array(image.dataobj)
#     image = np.moveaxis(image, [0, 1], [1, 0])
#     img_scale = tuple([img_size[i]/image.shape[i] for i in range(len(image.shape))])
#     image = zoom(image, img_scale)
#     # image = image / 255.
#     image = torch.from_numpy(image)
#     image = image.unsqueeze(0)  # Add channel dimension
#     # No need to add batch dimension here; will adjust when calling the model
#     return image

def preprocess(input_path):
    
    img_size=(137, 128, 128)
    image = ni.load(input_path)
    image = np.array(image.dataobj)
    image = np.moveaxis(image, [0, 1], [1, 0])
    img_scale = tuple([img_size[i]/image.shape[i] for i in range(len(image.shape))])
    image = zoom(image, img_scale)
    image = torch.from_numpy(image)
    image = image.unsqueeze(0).unsqueeze(0)
    image = image.to(device)
    
    return image

# Getting Latents from VAE

In [29]:
# Adjusted for processing multiple files
filenames = [i for i in os.listdir(imagePath) if i.endswith(".nii")]
filenames = sorted(filenames)
model_AE.eval()  # Make sure the model is in evaluation mode
latents = []
PT_500 = []
PT_4000 = []
fileName = None


for filename in filenames:
    # img = preprocess_image(path, filename)
    fullPath = os.path.join(imagePath, filename)
    img = preprocess(fullPath)
    
    # Forward pass through the model
    with torch.no_grad():
        z, latent = model_AE(img)
        # latent = model_AE(img)
        #z.append(z_log_sigma)
    
    # Directly use z_mean as the latent representation
    # latent = z_mean.detach().cpu().numpy().flatten()  # Flatten or adjust according to your needs
    # Extract labels using filename manipulation
    filename_ = filename.split('_')[0]
    fl = filename_[:5] + '_' + filename_[5:]
    pt500_val = labels[labels['ID'] == fl]['PT500'].iloc[0] if not labels[labels['ID'] == fl].empty else None
    pt4000_val = labels[labels['ID'] == fl]['PT4000'].iloc[0] if not labels[labels['ID'] == fl].empty else None
    
    if pt500_val is not None and pt4000_val is not None:
        PT_500.append(pt500_val)
        PT_4000.append(pt4000_val)
        latents.append(latent.detach().cpu().numpy())
        

# Remove the break to process all files


In [30]:
filenames

['smwp10001_T1.nii',
 'smwp10002_T1.nii',
 'smwp10003_T1.nii',
 'smwp10004_T1.nii',
 'smwp10005_T1.nii',
 'smwp10006_T1.nii',
 'smwp10007_T1.nii',
 'smwp10008_T1.nii',
 'smwp10009_T1.nii',
 'smwp10010_T1.nii',
 'smwp10011_T1.nii',
 'smwp10012_T1.nii',
 'smwp10013_T1.nii',
 'smwp10014_T1.nii',
 'smwp10015_T1.nii',
 'smwp10016_T1.nii',
 'smwp10017_T1.nii',
 'smwp10018_T1.nii',
 'smwp10019_T1.nii',
 'smwp10020_T1.nii',
 'smwp10021_T1.nii',
 'smwp10022_T1.nii',
 'smwp10023_T1.nii',
 'smwp10024_T1.nii',
 'smwp10025_T1.nii',
 'smwp10026_T1.nii',
 'smwp10027_T1.nii',
 'smwp10028_T1.nii',
 'smwp10029_T1.nii',
 'smwp10030_T1.nii',
 'smwp10031_T1.nii',
 'smwp10032_T1.nii',
 'smwp10033_T1.nii',
 'smwp10034_T1.nii',
 'smwp10035_T1.nii',
 'smwp10036_T1.nii',
 'smwp10037_T1.nii',
 'smwp10038_T1.nii',
 'smwp10039_T1.nii',
 'smwp10040_T1.nii',
 'smwp10041_T1.nii',
 'smwp10042_T1.nii',
 'smwp10043_T1.nii',
 'smwp10044_T1.nii',
 'smwp10045_T1.nii',
 'smwp10046_T1.nii',
 'smwp10047_T1.nii',
 'smwp10048_T

In [31]:
latents[0] == latents[1]

array([[False,  True, False,  True,  True, False,  True,  True, False,
        False, False, False,  True, False, False,  True,  True, False,
        False,  True,  True, False,  True,  True,  True, False, False,
         True, False,  True,  True,  True, False,  True,  True,  True,
         True,  True,  True, False,  True, False, False,  True,  True,
         True,  True, False,  True,  True,  True, False,  True,  True,
         True,  True, False, False,  True, False,  True,  True, False,
         True,  True,  True,  True, False,  True, False,  True,  True,
        False, False,  True,  True,  True, False,  True,  True,  True,
        False, False, False, False, False,  True,  True,  True,  True,
        False,  True,  True,  True, False,  True, False,  True,  True,
        False, False,  True, False,  True, False,  True, False, False,
         True,  True, False, False, False, False,  True,  True,  True,
        False,  True, False,  True,  True, False,  True,  True,  True,
      

# TEST TRAIN SPLIT

In [32]:
train_filenames = [
'smwp10067_T1.nii'
,'smwp10051_T1.nii'
,'smwp10071_T1.nii'
,'smwp10074_T1.nii'
,'smwp10123_T1.nii'
,'smwp10066_T1.nii'
,'smwp10058_T1.nii'
,'smwp10148_T1.nii'
,'smwp10155_T1.nii'
,'smwp10076_T1.nii'
,'smwp10002_T1.nii'
,'smwp10124_T1.nii'
,'smwp10108_T1.nii'
,'smwp10151_T1.nii'
,'smwp10042_T1.nii'
,'smwp10117_T1.nii'
,'smwp10046_T1.nii'
,'smwp10092_T1.nii'
,'smwp10061_T1.nii'
,'smwp10088_T1.nii'
,'smwp10162_T1.nii'
,'smwp10165_T1.nii'
,'smwp10040_T1.nii'
,'smwp10113_T1.nii'
,'smwp10028_T1.nii'
,'smwp10116_T1.nii'
,'smwp10024_T1.nii'
,'smwp10118_T1.nii'
,'smwp10095_T1.nii'
,'smwp10016_T1.nii'
,'smwp10039_T1.nii'
,'smwp10049_T1.nii'
,'smwp10126_T1.nii'
,'smwp10159_T1.nii'
,'smwp10070_T1.nii'
,'smwp10130_T1.nii'
,'smwp10004_T1.nii'
,'smwp10132_T1.nii'
,'smwp10008_T1.nii'
,'smwp10005_T1.nii'
,'smwp10093_T1.nii'
,'smwp10035_T1.nii'
,'smwp10169_T1.nii'
,'smwp10122_T1.nii'
,'smwp10125_T1.nii'
,'smwp10021_T1.nii'
,'smwp10153_T1.nii'
,'smwp10033_T1.nii'
,'smwp10012_T1.nii'
,'smwp10025_T1.nii'
,'smwp10131_T1.nii'
,'smwp10014_T1.nii'
,'smwp10141_T1.nii'
,'smwp10007_T1.nii'
,'smwp10069_T1.nii'
,'smwp10087_T1.nii'
,'smwp10102_T1.nii'
,'smwp10086_T1.nii'
,'smwp10072_T1.nii'
,'smwp10083_T1.nii'
,'smwp10101_T1.nii'
,'smwp10163_T1.nii'
,'smwp10171_T1.nii'
,'smwp10010_T1.nii'
,'smwp10115_T1.nii'
,'smwp10112_T1.nii'
,'smwp10157_T1.nii'
,'smwp10062_T1.nii'
,'smwp10094_T1.nii'
,'smwp10136_T1.nii'
,'smwp10097_T1.nii'
,'smwp10037_T1.nii'
,'smwp10146_T1.nii'
,'smwp10011_T1.nii'
,'smwp10129_T1.nii'
,'smwp10045_T1.nii'
,'smwp10104_T1.nii'
,'smwp10057_T1.nii'
,'smwp10152_T1.nii'
,'smwp10001_T1.nii'
,'smwp10023_T1.nii'
,'smwp10013_T1.nii'
,'smwp10009_T1.nii'
,'smwp10075_T1.nii'
,'smwp10127_T1.nii'
,'smwp10144_T1.nii'
,'smwp10017_T1.nii'
,'smwp10022_T1.nii'
,'smwp10114_T1.nii'
,'smwp10119_T1.nii'
,'smwp10044_T1.nii'
,'smwp10156_T1.nii'
,'smwp10050_T1.nii'
,'smwp10142_T1.nii'
,'smwp10019_T1.nii'
,'smwp10081_T1.nii'
,'smwp10077_T1.nii'
,'smwp10064_T1.nii'
,'smwp10048_T1.nii'
,'smwp10026_T1.nii'
,'smwp10018_T1.nii'
,'smwp10170_T1.nii'
,'smwp10060_T1.nii'
,'smwp10031_T1.nii'
,'smwp10030_T1.nii'
,'smwp10138_T1.nii'
,'smwp10168_T1.nii'
,'smwp10134_T1.nii'
,'smwp10043_T1.nii'
,'smwp10145_T1.nii'
,'smwp10079_T1.nii'
,'smwp10054_T1.nii'
,'smwp10161_T1.nii'
,'smwp10103_T1.nii'
,'smwp10073_T1.nii'
,'smwp10099_T1.nii'
,'smwp10041_T1.nii'
,'smwp10065_T1.nii'
,'smwp10109_T1.nii'
]

val_filenames = [
'smwp10034_T1.nii'
,'smwp10111_T1.nii'
,'smwp10003_T1.nii'
,'smwp10098_T1.nii'
,'smwp10029_T1.nii'
,'smwp10096_T1.nii'
,'smwp10006_T1.nii'
,'smwp10027_T1.nii'
,'smwp10059_T1.nii'
,'smwp10166_T1.nii'
,'smwp10147_T1.nii'
,'smwp10082_T1.nii'
,'smwp10143_T1.nii'
,'smwp10110_T1.nii'
,'smwp10139_T1.nii'
,'smwp10167_T1.nii'
,'smwp10089_T1.nii'
,'smwp10154_T1.nii'
,'smwp10100_T1.nii'
,'smwp10084_T1.nii'
,'smwp10036_T1.nii'
,'smwp10032_T1.nii'
,'smwp10150_T1.nii'
,'smwp10160_T1.nii'
,'smwp10090_T1.nii'
,'smwp10015_T1.nii'
,'smwp10106_T1.nii'
,'smwp10055_T1.nii'
,'smwp10164_T1.nii'
,'smwp10038_T1.nii'
,'smwp10120_T1.nii'
,'smwp10053_T1.nii'
,'smwp10020_T1.nii'
,'smwp10133_T1.nii'
,'smwp10085_T1.nii'
,'smwp10140_T1.nii'
,'smwp10080_T1.nii'
,'smwp10056_T1.nii'
,'smwp10158_T1.nii'
,'smwp10135_T1.nii'
,'smwp10107_T1.nii'
,'smwp10091_T1.nii'
,'smwp10128_T1.nii'
,'smwp10063_T1.nii'
,'smwp10137_T1.nii'
,'smwp10068_T1.nii'
,'smwp10105_T1.nii'
,'smwp10052_T1.nii'
,'smwp10078_T1.nii'
,'smwp10149_T1.nii'
,'smwp10047_T1.nii'
,'smwp10121_T1.nii'
]

# Map filenames to their corresponding indices in the full dataset
filename_to_index = {fname: i for i, fname in enumerate(filenames)}

# Extract indices for training and validation sets
train_indices = [filename_to_index[fname] for fname in train_filenames]
val_indices = [filename_to_index[fname] for fname in val_filenames]

# latents = np.array(latents)  # Convert to NumPy array
latents_array = np.array(latents).reshape(len(latents), -1)

PT_500_np = np.array(PT_500)  # Convert to NumPy array

# Use indices to extract the data for training and validation sets
X_train = latents_array[train_indices]
y_train = PT_500_np[train_indices]
X_test = latents_array[val_indices]
y_test = PT_500_np[val_indices]

In [33]:
PT_4000_np = np.array(PT_4000)  # Convert to NumPy array

# Use indices to extract the data for training and validation sets
X_train_4000 = latents_array[train_indices]
y_train_4000 = PT_4000_np[train_indices]
X_test_4000 = latents_array[val_indices]
y_test_4000 = PT_4000_np[val_indices]

# Random Forest on Latents

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

latents_array = np.array(latents).reshape(len(latents), -1)
PT_500_np = np.array(PT_500) 
PT_4000_np = np.array(PT_4000) 


# Splitting the latent representations and their corresponding numerical values into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(latents_array, PT_500_np, test_size=0.3, random_state=42)

# Train RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train, y_train)

# Evaluation
predictions = regressor.predict(X_test)
print(predictions)
mse = mean_squared_error(y_test, predictions)
print(f'Random Forest MSE: {mse}')

# Splitting the latent representations and their corresponding numerical values into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(latents_array, PT_4000_np, test_size=0.3, random_state=42)

# Train RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(X_train_4000, y_train_4000)

# Evaluation
predictions = regressor.predict(X_test_4000)
print(predictions)
mse = mean_squared_error(y_test_4000, predictions)
print(f'Random Forest MSE: {mse}')


[14.23 10.9  13.79 25.67 17.55 12.08  6.51 12.08  7.7  13.77 11.02 22.2
 12.61 18.28 13.13 14.57 10.65  9.11 13.04 18.97  7.64 14.51 12.19 16.06
 18.05  8.87 12.65 12.03 13.44  7.27 10.79 12.76  7.82 11.48 10.73 10.74
 14.48 17.19 14.63 12.35  8.75 14.08 12.04 11.25 15.19 10.75  8.41  8.22
 11.36  6.73 11.43 10.87]
Random Forest MSE: 87.91819230769232
[32.29 19.43 45.88 48.18 36.04 32.2  31.79 23.36 19.58 42.48 23.73 44.85
 29.18 43.46 27.15 47.4  24.92 31.05 34.28 50.47 25.24 44.68 27.27 35.67
 32.3  34.07 28.85 34.84 38.17 24.41 33.62 41.76 25.45 46.13 42.2  32.01
 37.77 42.28 25.22 45.89 26.33 34.67 42.06 47.35 31.03 24.58 23.45 25.49
 25.3  27.07 31.43 25.39]
Random Forest MSE: 484.9615153846154


# XGB on Latents

In [35]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

latents_array = np.array(latents).reshape(len(latents), -1)  # Your flattened latent vectors
PT_500_np = np.array(PT_500)  # PT_500 numerical values
PT_4000_np = np.array(PT_4000)  # PT_4000 numerical values

# Predicting PT_500 values using XGBoost
# X_train, X_test, y_train, y_test = train_test_split(latents_array, PT_500_np, test_size=0.3, random_state=42)

xgb_regressor = xgb.XGBRegressor(n_estimators=100, objective='reg:squarederror')
xgb_regressor.fit(X_train, y_train)

predictions = xgb_regressor.predict(X_test)
print("ground truth",y_test)
print(predictions)

mse = mean_squared_error(y_test, predictions)
print(f'XGBoost MSE for PT_500: {mse}')

# Predicting PT_4000 values using XGBoost
# X_train, X_test, y_train, y_test = train_test_split(latents_array, PT_4000_np, test_size=0.3, random_state=42)

xgb_regressor = xgb.XGBRegressor(n_estimators=100, objective='reg:squarederror')
xgb_regressor.fit(X_train_4000, y_train_4000)

predictions = xgb_regressor.predict(X_test_4000)
print("ground truth",y_test_4000)
print(predictions)
mse = mean_squared_error(y_test_4000, predictions)
print(f'XGBoost MSE for PT_4000: {mse}')


ground truth [ 5  5  5 10 13 23 10 -5  8  3 15  5  0 13  8  0 25 10 10 18  5  8 10  0
 25 13  8  3 18 20 10 23  0 13  8  5 15  5  3 13  9  0 38 15 20 20  0 -3
  3 20  8  8]
[12.171845   8.574337  22.992125  28.139708  19.173525  14.494497
  8.512958   9.135817   5.9693723 11.157347  12.2764015 19.729847
 10.261713  12.020353  10.392263  14.680391   9.087205  13.104547
 11.425393  24.539078  13.62485    9.414397   4.1595197  9.837781
 13.120654   9.0325365 14.6752205  4.5429487 11.484718   7.8316965
 12.402602  10.440407   7.7590656 16.318987   8.429059   9.79118
  6.3139925 12.814491  13.59283   13.953571   3.664246  14.316917
 14.1813965  9.6992855  6.4567213 15.177594   7.280269   9.985841
  2.3351972  9.1552    10.625794   9.174586 ]
XGBoost MSE for PT_500: 85.62056206042345
ground truth [78  5 45 20 13 48 18 -3  5  3 55 10  8 25 23 28 35 23  3 45 15  5 48  5
 48 53 10 23 53 58 20 35 -3 63  5 20 18 15 30 30 55 45 40 45 18 60 -3 13
 45 28 10 43]
[23.19543   6.839634 38.217842 57.7206

# MNN Function

In [36]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)  # Dropout layer after first ReLU
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)  # Dropout layer after second ReLU
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x, feature_extract=False):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        if feature_extract:
            # Return features just before the last dropout for feature extraction
            return x
        x = self.fc3(x)
        return x

    def extract_features(self, dataloader):
        self.eval()  # Set model to evaluation mode
        features = []
        with torch.no_grad():
            for inputs, _ in dataloader:
                outputs = self(inputs, feature_extract=True)
                features.append(outputs)
        return torch.cat(features, 0)


# MNN (2 hidden layers) on Latents on PT500

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math


# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(latents_array, PT_500_np, test_size=0.3, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Use the same scaler to transform the test set

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Add extra dimension for targets

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # Shuffle=False for testing

    
# Instantiate the model
dropout_rate = 0.5  # Dropout rate can be adjusted as needed
model = SimpleNN(input_size=X_train_tensor.shape[1], hidden_size1=64, hidden_size2=32, output_size=1, dropout_rate=dropout_rate)
model.train()

l2_lambda = 0.01  # L2 regularization strength
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)  # L2 regularization is added via weight_decay
criterion = nn.MSELoss()

train_features = model.extract_features(train_dataloader)
test_features = model.extract_features(test_dataloader)

X_train_features = train_features.numpy()
X_test_features = test_features.numpy()

# Use MSE Loss and the Adam optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
# Testing phase
model.eval()  # Set the model to evaluation mode
test_losses = []
all_targets = []  # List to store all the ground truth values
all_predictions = []  # List to store all the model's predictions

with torch.no_grad():
    for inputs, targets in test_dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_losses.append(loss.item())
        
        all_targets.extend(targets.view(-1).tolist())  # Flatten and convert to list
        all_predictions.extend(outputs.view(-1).tolist())  # Flatten and convert to list

# Calculate average loss across all test batches
average_test_loss = sum(test_losses) / len(test_losses)
test_rmse = math.sqrt(average_test_loss)
print(f'Test RMSE: {test_rmse:.4f}')

# Display ground truth vs. predictions
for i in range(len(all_targets)):
    print(f'Ground Truth: {all_targets[i]:.4f}, Prediction: {all_predictions[i]:.4f}')

Epoch [1/100], Loss: 181.8996
Epoch [11/100], Loss: 153.0902
Epoch [21/100], Loss: 182.1541
Epoch [31/100], Loss: 88.4302
Epoch [41/100], Loss: 113.6936
Epoch [51/100], Loss: 36.3566
Epoch [61/100], Loss: 156.6059
Epoch [71/100], Loss: 73.9263
Epoch [81/100], Loss: 88.2122
Epoch [91/100], Loss: 39.7692
Test RMSE: 9.7935
Ground Truth: 5.0000, Prediction: 11.2609
Ground Truth: 5.0000, Prediction: 0.9547
Ground Truth: 5.0000, Prediction: 14.9995
Ground Truth: 10.0000, Prediction: 28.4357
Ground Truth: 13.0000, Prediction: 17.2401
Ground Truth: 23.0000, Prediction: 15.0993
Ground Truth: 10.0000, Prediction: 7.4128
Ground Truth: -5.0000, Prediction: 6.6912
Ground Truth: 8.0000, Prediction: 6.9261
Ground Truth: 3.0000, Prediction: 15.6006
Ground Truth: 15.0000, Prediction: 12.0102
Ground Truth: 5.0000, Prediction: 26.5010
Ground Truth: 0.0000, Prediction: 14.7120
Ground Truth: 13.0000, Prediction: 14.1091
Ground Truth: 8.0000, Prediction: 13.1790
Ground Truth: 0.0000, Prediction: 19.4183
Gro

# Ensemble of MNN and XGBoost

In [38]:
from xgboost import XGBRegressor

# Instantiate the model
model = XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, subsample=0.8, colsample_bytree=0.8, objective='reg:squarederror')

# Fit the model
model.fit(X_train_features, y_train)

# Make predictions
predictions = model.predict(X_test_features)

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(y_test, predictions))
print(f"Test RMSE: {rmse:.4f}")
for i, (gt, pred) in enumerate(zip(y_test, predictions)):
    print(f'Ground Truth: {gt:.4f}, Prediction: {pred:.4f}')

Test RMSE: 10.6307
Ground Truth: 5.0000, Prediction: 24.7490
Ground Truth: 5.0000, Prediction: 11.6028
Ground Truth: 5.0000, Prediction: 15.1078
Ground Truth: 10.0000, Prediction: 11.9791
Ground Truth: 13.0000, Prediction: 11.6285
Ground Truth: 23.0000, Prediction: 11.1188
Ground Truth: 10.0000, Prediction: 16.8362
Ground Truth: -5.0000, Prediction: 17.9167
Ground Truth: 8.0000, Prediction: 15.7339
Ground Truth: 3.0000, Prediction: 18.1512
Ground Truth: 15.0000, Prediction: 14.6219
Ground Truth: 5.0000, Prediction: 11.8281
Ground Truth: 0.0000, Prediction: 10.8780
Ground Truth: 13.0000, Prediction: 11.8248
Ground Truth: 8.0000, Prediction: 7.5692
Ground Truth: 0.0000, Prediction: 14.9015
Ground Truth: 25.0000, Prediction: 12.0865
Ground Truth: 10.0000, Prediction: 9.9673
Ground Truth: 10.0000, Prediction: 9.0826
Ground Truth: 18.0000, Prediction: 15.6674
Ground Truth: 5.0000, Prediction: 15.1696
Ground Truth: 8.0000, Prediction: 14.4954
Ground Truth: 10.0000, Prediction: 12.7026
Ground

# MNN (2 hidden layers) on Latents on PT4000

In [39]:
X_train, X_test, y_train, y_test = X_train_4000, X_test_4000, y_train_4000, y_test_4000

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(latents_array, PT_4000_np, test_size=0.3, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Use the same scaler to transform the test set

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Add extra dimension for targets

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # Shuffle=False for testing
    
# Instantiate the model
dropout_rate = 0.5  # Dropout rate can be adjusted as needed
model = SimpleNN(input_size=X_train_tensor.shape[1], hidden_size1=64, hidden_size2=32, output_size=1, dropout_rate=dropout_rate)
model.train()

l2_lambda = 0.01  # L2 regularization strength
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=l2_lambda)  # L2 regularization is added via weight_decay
criterion = nn.MSELoss()

train_features = model.extract_features(train_dataloader)
test_features = model.extract_features(test_dataloader)

X_train_features = train_features.numpy()
X_test_features = test_features.numpy()


# Training loop
epochs = 100
for epoch in range(epochs):
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
# Testing phase
model.eval()  # Set the model to evaluation mode
test_losses = []
all_targets = []  # List to store all the ground truth values
all_predictions = []  # List to store all the model's predictions

with torch.no_grad():
    for inputs, targets in test_dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_losses.append(loss.item())
        
        all_targets.extend(targets.view(-1).tolist())  # Flatten and convert to list
        all_predictions.extend(outputs.view(-1).tolist())  # Flatten and convert to list

# Calculate average loss across all test batches
average_test_loss = sum(test_losses) / len(test_losses)
test_rmse = math.sqrt(average_test_loss)
print(f'Test RMSE: {test_rmse:.4f}')

# Display ground truth vs. predictions
for i in range(len(all_targets)):
    print(f'Ground Truth: {all_targets[i]:.4f}, Prediction: {all_predictions[i]:.4f}')

Epoch [1/100], Loss: 1475.6288
Epoch [11/100], Loss: 1571.0656
Epoch [21/100], Loss: 617.1367
Epoch [31/100], Loss: 622.1299
Epoch [41/100], Loss: 604.7731
Epoch [51/100], Loss: 361.0830
Epoch [61/100], Loss: 361.5636
Epoch [71/100], Loss: 373.6135
Epoch [81/100], Loss: 221.1975
Epoch [91/100], Loss: 416.6325
Test RMSE: 25.8575
Ground Truth: 78.0000, Prediction: 34.5438
Ground Truth: 5.0000, Prediction: -3.8884
Ground Truth: 45.0000, Prediction: 39.1312
Ground Truth: 20.0000, Prediction: 67.8551
Ground Truth: 13.0000, Prediction: 25.8748
Ground Truth: 48.0000, Prediction: 35.1492
Ground Truth: 18.0000, Prediction: 26.2763
Ground Truth: -3.0000, Prediction: 19.7647
Ground Truth: 5.0000, Prediction: 17.9533
Ground Truth: 3.0000, Prediction: 41.5584
Ground Truth: 55.0000, Prediction: 21.1442
Ground Truth: 10.0000, Prediction: 80.1423
Ground Truth: 8.0000, Prediction: 36.5891
Ground Truth: 25.0000, Prediction: 37.9988
Ground Truth: 23.0000, Prediction: 27.1078
Ground Truth: 28.0000, Predic

# Ensemble of MNN and XGBoost

In [21]:
from xgboost import XGBRegressor

# Instantiate the model
model = XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, subsample=0.8, colsample_bytree=0.8, objective='reg:squarederror')

# Fit the model
model.fit(X_train_features, y_train)

# Make predictions
predictions = model.predict(X_test_features)

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(y_test, predictions))
print(f"Test RMSE: {rmse:.4f}")
for i, (gt, pred) in enumerate(zip(y_test, predictions)):
    print(f'Ground Truth: {gt:.4f}, Prediction: {pred:.4f}')

Test RMSE: 23.3541
Ground Truth: 78.0000, Prediction: 26.1377
Ground Truth: 5.0000, Prediction: 38.9419
Ground Truth: 45.0000, Prediction: 47.0229
Ground Truth: 20.0000, Prediction: 46.4734
Ground Truth: 13.0000, Prediction: 46.2009
Ground Truth: 48.0000, Prediction: 22.5880
Ground Truth: 18.0000, Prediction: 29.0854
Ground Truth: -3.0000, Prediction: 30.8086
Ground Truth: 5.0000, Prediction: 29.0428
Ground Truth: 3.0000, Prediction: 21.6108
Ground Truth: 55.0000, Prediction: 33.7466
Ground Truth: 10.0000, Prediction: 51.8923
Ground Truth: 8.0000, Prediction: 18.4383
Ground Truth: 25.0000, Prediction: 34.4669
Ground Truth: 23.0000, Prediction: 22.5806
Ground Truth: 28.0000, Prediction: 36.9611
Ground Truth: 35.0000, Prediction: 42.5925
Ground Truth: 23.0000, Prediction: 30.4321
Ground Truth: 3.0000, Prediction: 43.2701
Ground Truth: 45.0000, Prediction: 42.1412
Ground Truth: 15.0000, Prediction: 19.4155
Ground Truth: 5.0000, Prediction: 42.4016
Ground Truth: 48.0000, Prediction: 48.845

# CNN

In [22]:
class SimpleCNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)  # Dropout layer after first ReLU
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)  # Dropout layer after second ReLU
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.model = model_VAE()

    def forward(self, img, feature_extract=False):
        
        x = self.model(img)
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        if feature_extract:
            # Return features just before the last dropout for feature extraction
            return x
        x = self.fc3(x)
        return x

    def extract_features(self, dataloader):
        self.eval()  # Set model to evaluation mode
        features = []
        with torch.no_grad():
            for inputs, _ in dataloader:
                outputs = self(inputs, feature_extract=True)
                features.append(outputs)
        return torch.cat(features, 0)


In [ ]:
# # Training loop
# epochs = 100
# for epoch in range(epochs):
#     for inputs, targets in train_dataloader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()
    
#     if epoch % 10 == 0:
#         print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
# # Testing phase
# model.eval()  # Set the model to evaluation mode
# test_losses = []
# all_targets = []  # List to store all the ground truth values
# all_predictions = []  # List to store all the model's predictions

# with torch.no_grad():
#     for inputs, targets in test_dataloader:
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         test_losses.append(loss.item())
        
#         all_targets.extend(targets.view(-1).tolist())  # Flatten and convert to list
#         all_predictions.extend(outputs.view(-1).tolist())  # Flatten and convert to list

# # Calculate average loss across all test batches
# average_test_loss = sum(test_losses) / len(test_losses)
# test_rmse = math.sqrt(average_test_loss)
# print(f'Test RMSE: {test_rmse:.4f}')

# # Display ground truth vs. predictions
# for i in range(len(all_targets)):
#     print(f'Ground Truth: {all_targets[i]:.4f}, Prediction: {all_predictions[i]:.4f}')